### Okt로 어휘사전 만들기

In [57]:
from konlpy.tag import Okt
import string
import pandas as pd
import re

Okt = Okt()

file3 = pd.read_csv('../DATA/king_3.csv', encoding='utf-8', sep=';')

with open('../DATA/stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = f.readlines()
stop_words = [x.strip() for x in stop_words]
# print(stop_words)


In [24]:

vocab = {}
data = file3['text'].values
sentences = []
for line in data:
    # 토크나이징
    tokens = Okt.nouns(line)
    # 정규표현식으로 한글만 추출
    tokens = [token for token in tokens if re.match(r'^[가-힣]+$', token)]
    
    # 불용어 제거
    tokens = [token for token in tokens if token not in stop_words]
    
    # 빈도수 계산
    for token in tokens:
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1
    # 데이터프레임에 추가
    sentences.append(tokens)

vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

In [25]:
print('vocab size:', len(vocab))
# vocab 일부만 출력
i = 0
for word, freq in vocab:
    print(f'[{word}: {freq}]', end=', ')
    i += 1
    if i > 100:
        break

vocab size: 29476
[말: 24436], [사람: 17938], [하니: 16808], [임금: 12539], [수: 10977], [명: 10277], [은: 9165], [청: 7035], [지금: 6769], [신: 6623], [그대로: 6610], [죄: 6576], [도: 6503], [여러: 6314], [관: 6242], [고: 5944], [백성: 5728], [전하: 5670], [내: 5670], [이제: 5473], [전: 5465], [생각: 5389], [알: 5386], [법: 5291], [의논: 5123], [위: 5086], [나라: 5009], [백: 4729], [행: 4720], [다시: 4705], [두: 4633], [뒤: 4606], [하소: 4449], [번: 4441], [정: 4296], [곳: 4286], [계: 4245], [중: 4189], [가지: 4177], [함: 4152], [마음: 4019], [못: 3941], [인도: 3897], [사신: 3870], [뜻: 3821], [예조: 3784], [인: 3776], [하라: 3763], [자가: 3761], [데: 3700], [제사: 3617], [자리: 3595], [보고: 3462], [앞: 3449], [필: 3390], [설치: 3370], [때문: 3362], [품: 3356], [군사: 3292], [고을: 3205], [상: 3107], [감사: 3040], [세: 3036], [병조: 2990], [안: 2917], [전지: 2877], [좌: 2865], [의정부: 2819], [서로: 2760], [거: 2757], [종: 2726], [집: 2669], [정사: 2623], [바: 2585], [수령: 2570], [찬: 2508], [몸: 2481], [후: 2420], [술: 2396], [온: 2341], [이하: 2258], [부: 2252], [의거: 2235], [향: 2219], [장: 2203], [판

In [12]:
# vocab.txt 저장
with open('../DATA/vocab/vocab3.txt', 'w', encoding='utf-8') as f:
    for word, freq in vocab:
        f.write(f'{word}: {freq}\n')

In [28]:
# vocab 불러오기
with open('../DATA/vocab/vocab3.txt', 'r', encoding='utf-8') as f:
    vocabs = f.readlines()
    # dict로 변환
    vocabs = {x.split(':')[0]: int(x.split(':')[1]) for x in vocab}
    print(vocabs)

AttributeError: 'tuple' object has no attribute 'split'

In [27]:
# vocab에서 글자수가 1개인 단어 제거
vocab_no1 = [x for x in vocab if len(x.split(':')[0]) > 1]
print(len(vocab_no1))

AttributeError: 'tuple' object has no attribute 'split'

In [ ]:
# 시각화 : 워드클라우드
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf', background_color='white', width=800, height=600, colormap='tab20b')
cloud = wc.generate_from_frequencies(dict(vocab))
plt.figure(figsize=(10, 8))
plt.axis('off')
plt.imshow(cloud)
plt.show()

In [ ]:
file3['tokenized'] = sentences
file3.head()

In [ ]:
file3.to_csv('../DATA/okt/king_3_tokenized.csv', encoding='utf-8', sep=';')

### 임베딩 함수 사용 : EmbeddingBag

In [ ]:
# %conda install -c pytorch torchtext
# %conda update -n base -c defaults conda

In [58]:
for line in file3['text'].values:
    print(line)
    print(Okt.nouns(line))
    # iter() : 반복자 객체 생성
    print('---------------------------------')
    break

 지신사(知申事) 정흠지(鄭欽之)가 포천(抱川)에 가서 그의 어머니를 뵙고 오기를 청하니, 이를 허락하고 인하여 술과 고기를 내려 주었다.
['신사', '정흠', '포천', '그', '어머니', '오기', '청', '허락', '술', '고기']
---------------------------------


In [69]:
# 이터레이터 토큰화 함수
from konlpy.tag import Okt
from torchtext.vocab import build_vocab_from_iterator

Okt = Okt()

def yield_tokens(data_iter):
    for text in data_iter:
        # print(text)
        yield iter(Okt.nouns(text))
        
# next(yield_tokens(file3['text'])) # 확인함

# 파일 읽기
file3 = pd.read_csv('../DATA/king_3.csv', encoding='utf-8', sep=';')
data = file3['text'].values

# vocab 생성
vocab = build_vocab_from_iterator(yield_tokens(data), specials=["<unk>"])
# vocab 확인
print(vocab.get_itos()[:10])

# <unk> 인덱스를 0으로 설정
vocab.set_default_index(vocab["<unk>"])

['<unk>', '것', '이', '의', '그', '말', '를', '사람', '하니', '등']


In [73]:
# 불용어 제거
with open('../DATA/stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = f.readlines()
stop_words = [x.strip() for x in stop_words]
stop_words += ['하니', '수', '은', '지금', '그대로', '도', '여러', '관', '고', '내', 이제, 전, 생각]

vocab_no_stop = [x for x in vocab.get_itos() if x not in stop_words]
print(len(vocab_no_stop))
print(vocab_no_stop[:20])

# 글자수가 1개인 단어 제거
vocab_no1 = [x for x in vocab.get_itos() if len(x) > 1]
print(len(vocab_no1))
print(vocab_no1[:20])

29467
['<unk>', '말', '사람', '임금', '명', '청', '신', '죄', '백성', '전하', '이제', '전', '생각', '알', '법', '의논', '위', '나라', '백', '행']
28897
['<unk>', '사람', '하니', '임금', '모두', '지금', '그대로', '여러', '비록', '백성', '전하', '어찌', '이제', '생각', '의논', '나라', '또한', '우리', '다시', '하소']


In [70]:
# 예시
vocab['사람'] # 487

7

In [64]:
# vocab 확인
len(vocab.get_itos())

# 빈도수 확인

AttributeError: 'Vocab' object has no attribute 'freqs'

In [ ]:
# # EmbeddingBag을 포함한 RNN 모델
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # 데이터셋 클래스
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, data, labels):
#         # data = file3['tokenized'].values; labels = file3['label'].values
#         self.data = 
#         self.labels = labels
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         return self.data[idx], self.labels[idx]
